In [1]:
import pickle
import random
import numpy
import torch
import os

from Auxiliary_Functions.generate_observation import generate_observations, split_dataset
from Auxiliary_Functions.save_progress import save_progress
from Auxiliary_Functions.matrix_normalization import normalize_max_row_norm

from Denoising_Algorithms.Projectors.DnCNN import DnCNN
from Denoising_Algorithms.DL_Training.evaluation import average_PSNR
from Denoising_Algorithms.DL_Training.loss_functions import LastLayerLoss
from Denoising_Algorithms.DL_Training.training import train_main
from Denoising_Algorithms.DAMP_Network.DAMP import DAMP
from Denoising_Algorithms.DL_Training.evaluation import average_PSNR

In [2]:
current_trial = 0

torch.manual_seed(current_trial)
numpy.random.seed(current_trial)
random.seed(current_trial)
torch.cuda.manual_seed(current_trial)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
with open('Auxiliary_Functions/sensing_matrices/50 x 50 images/Gaussian_10.pkl', 'rb') as f:
    gaussian_10 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/50 x 50 images/Gaussian_20.pkl', 'rb') as f:
    gaussian_20 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/50 x 50 images/Gaussian_30.pkl', 'rb') as f:
    gaussian_30 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/50 x 50 images/Gaussian_40.pkl', 'rb') as f:
    gaussian_40 = pickle.load(f)

In [4]:
with open('ILSVRC2012_50x50_small.pkl', 'rb') as f:
    ILSVRC2012 = pickle.load(f)

# We only work with the first 25000 images
img_dataset = ILSVRC2012[:25000]

In [5]:
gaussian_10_normalized = normalize_max_row_norm(gaussian_10)
gaussian_20_normalized = normalize_max_row_norm(gaussian_20)
gaussian_30_normalized = normalize_max_row_norm(gaussian_30)
gaussian_40_normalized = normalize_max_row_norm(gaussian_40)

gaussian_10_observations = generate_observations(img_dataset, gaussian_10_normalized)
gaussian_20_observations = generate_observations(img_dataset, gaussian_20_normalized)
gaussian_30_observations = generate_observations(img_dataset, gaussian_30_normalized)
gaussian_40_observations = generate_observations(img_dataset, gaussian_40_normalized)

gaussian_10_train, gaussian_10_test = split_dataset(gaussian_10_observations, train_ratio = 0.9, seed = 0)
gaussian_20_train, gaussian_20_test = split_dataset(gaussian_20_observations, train_ratio = 0.9, seed = 0)
gaussian_30_train, gaussian_30_test = split_dataset(gaussian_30_observations, train_ratio = 0.9, seed = 0)
gaussian_40_train, gaussian_40_test = split_dataset(gaussian_40_observations, train_ratio = 0.9, seed = 0)

In [6]:
projector = DnCNN(depth = 5)

# Last Layer Loss
loss_function = LastLayerLoss()

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# 5 Projections
num_projections = 5

# Folder Path for 5 Projections
proj5_path = 'DAMP_No_Residual-LastLayer/5 Projections'

gauss_10_proj5_weights, gauss_10_proj5_hist = train_main(gaussian_10_normalized, [1] * num_projections, gaussian_10_train, model=projector, device=device, model_type='DAMP', split=0.2, 
                                                     numProjections=num_projections, residual=False, loss_function=loss_function, model_dir='Model_Paths/Gaussian_10_proj5', display=True)

gauss_20_proj5_weights, gauss_20_proj5_hist = train_main(gaussian_20_normalized, [1] * num_projections, gaussian_20_train, model=projector, device=device, model_type='DAMP', split=0.2, 
                                                     numProjections=num_projections, residual=False, loss_function=loss_function, model_dir='Model_Paths/Gaussian_20_proj5', display=True)

gauss_30_proj5_weights, gauss_30_proj5_hist = train_main(gaussian_30_normalized, [1] * num_projections, gaussian_30_train, model=projector, device=device, model_type='DAMP', split=0.2, 
                                                     numProjections=num_projections, residual=False, loss_function=loss_function, model_dir='Model_Paths/Gaussian_30_proj5', display=True)

gauss_40_proj5_weights, gauss_40_proj5_hist = train_main(gaussian_40_normalized, [1] * num_projections, gaussian_40_train, model=projector, device=device, model_type='DAMP', split=0.2, 
                                                     numProjections=num_projections, residual=False, loss_function=loss_function, model_dir='Model_Paths/Gaussian_40_proj5', display=True)

# Store Best Model Weights and History

save_progress(proj5_path, gauss_10_proj5_weights, gauss_10_proj5_hist, '10% Sampling')
save_progress(proj5_path, gauss_20_proj5_weights, gauss_20_proj5_hist, '20% Sampling')
save_progress(proj5_path, gauss_30_proj5_weights, gauss_30_proj5_hist, '30% Sampling')
save_progress(proj5_path, gauss_40_proj5_weights, gauss_40_proj5_hist, '40% Sampling')

Validation Size: 4500, Train Size: 18000
Epoch 1/300
Average training loss: 0.02895980746050676
Average test loss: 0.018552980792191292
Epoch 2/300
Average training loss: 0.015055556972821554
Average test loss: 0.03334287686480416
Epoch 3/300
Average training loss: 0.014060406862033739
Average test loss: 0.020443564309014214
Epoch 4/300
Average training loss: 0.013609955104688803
Average test loss: 0.018710224603613216
Epoch 5/300
Average training loss: 0.012931285546057754
Average test loss: 0.018182370210687318
Epoch 6/300
Average training loss: 0.012864478117889828
Average test loss: 0.027331066283914778
Epoch 7/300
Average training loss: 0.012601749579939578
Average test loss: 0.014087731497155295
Epoch 8/300
Average training loss: 0.012114678683380287
Average test loss: 0.01685232718785604
Epoch 9/300
Average training loss: 0.012103050229450067
Average test loss: 0.015599329979055457
Epoch 10/300
Average training loss: 0.011680418831606707
Average test loss: 0.015654376731978524
E

In [8]:
gauss_10_proj5_model = DAMP(gaussian_10_normalized, 0.01, num_projections, projector, device, residual=False).to(device)
gauss_20_proj5_model = DAMP(gaussian_20_normalized, 0.01, num_projections, projector, device, residual=False).to(device)
gauss_30_proj5_model = DAMP(gaussian_30_normalized, 0.01, num_projections, projector, device, residual=False).to(device)
gauss_40_proj5_model = DAMP(gaussian_40_normalized, 0.01, num_projections, projector, device, residual=False).to(device)

gauss_10_proj5_psnr = average_PSNR(gauss_10_proj5_model, gauss_10_proj5_weights, num_projections, gaussian_10_test, print_psnr=True)
gauss_20_proj5_psnr = average_PSNR(gauss_20_proj5_model, gauss_20_proj5_weights, num_projections, gaussian_20_test, print_psnr=True)
gauss_30_proj5_psnr = average_PSNR(gauss_30_proj5_model, gauss_30_proj5_weights, num_projections, gaussian_30_test, print_psnr=True)
gauss_40_proj5_psnr = average_PSNR(gauss_40_proj5_model, gauss_40_proj5_weights, num_projections, gaussian_40_test, print_psnr=True)

with open(os.path.join(proj5_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_10_proj5_psnr, f)
with open(os.path.join(proj5_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_20_proj5_psnr, f)
with open(os.path.join(proj5_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_30_proj5_psnr, f)
with open(os.path.join(proj5_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_40_proj5_psnr, f)

Average PSNR for Projection Layer 0 across 2500 images: -12.33
Average PSNR for Projection Layer 1 across 2500 images: 10.83
Average PSNR for Projection Layer 2 across 2500 images: 21.10
Average PSNR for Projection Layer 3 across 2500 images: 16.03
Average PSNR for Projection Layer 4 across 2500 images: 25.00
Average PSNR for Projection Layer 0 across 2500 images: -9.11
Average PSNR for Projection Layer 1 across 2500 images: 12.46
Average PSNR for Projection Layer 2 across 2500 images: 19.78
Average PSNR for Projection Layer 3 across 2500 images: 18.94
Average PSNR for Projection Layer 4 across 2500 images: 24.07


In [ ]:
# 15 Projections
num_projections = 15

# Folder Path for 15 Projections
proj15_path = 'DAMP_No_Residual-LastLayer/15 Projections'

gauss_10_proj15_weights, gauss_10_proj15_hist = train_main(gaussian_10_normalized, [1] * num_projections, gaussian_10_train, model=projector, device=device, model_type='DAMP', split=0.2, 
                                                     numProjections=num_projections, residual=False, loss_function=loss_function, model_dir='Model_Paths/Gaussian_10_proj15', display=True)

gauss_20_proj15_weights, gauss_20_proj15_hist = train_main(gaussian_20_normalized, [1] * num_projections, gaussian_20_train, model=projector, device=device, model_type='DAMP', split=0.2, 
                                                     numProjections=num_projections, residual=False, loss_function=loss_function, model_dir='Model_Paths/Gaussian_20_proj15', display=True)

gauss_30_proj15_weights, gauss_30_proj15_hist = train_main(gaussian_30_normalized, [1] * num_projections, gaussian_30_train, model=projector, device=device, model_type='DAMP', split=0.2, 
                                                     numProjections=num_projections, residual=False, loss_function=loss_function, model_dir='Model_Paths/Gaussian_30_proj15', display=True)

gauss_40_proj15_weights, gauss_40_proj15_hist = train_main(gaussian_40_normalized, [1] * num_projections, gaussian_40_train, model=projector, device=device, model_type='DAMP', split=0.2, 
                                                     numProjections=num_projections, residual=False, loss_function=loss_function, model_dir='Model_Paths/Gaussian_40_proj15', display=True)

# Store Best Model Weights and History

save_progress(proj15_path, gauss_10_proj15_weights, gauss_10_proj15_hist, '10% Sampling')
save_progress(proj15_path, gauss_20_proj15_weights, gauss_20_proj15_hist, '20% Sampling')
save_progress(proj15_path, gauss_30_proj15_weights, gauss_30_proj15_hist, '30% Sampling')
save_progress(proj15_path, gauss_40_proj15_weights, gauss_40_proj15_hist, '40% Sampling')

Validation Size: 4500, Train Size: 18000
Epoch 1/300
Average training loss: 0.025585786362489066
Average test loss: 0.01886017457313008
Epoch 2/300
Average training loss: 0.01473247568640444
Average test loss: 0.7783492579294575
Epoch 3/300
Average training loss: 0.013620455945531526
Average test loss: 0.20301939954111972
Epoch 4/300
Average training loss: 0.012823423234124979
Average test loss: 1.755157343480322
Epoch 5/300
Average training loss: 0.011734123920400937
Average test loss: 4.194762672040198
Epoch 6/300
Average training loss: 0.011423991880483098
Average test loss: 0.01581843978001012
Epoch 7/300
Average training loss: 0.011122138099537956
Average test loss: 0.9355832248942719
Epoch 8/300
Average training loss: 0.010512172136041853
Average test loss: 0.03599745808872912
Epoch 9/300
Average training loss: 0.010966295165320238
Average test loss: 0.7961201444698704
Epoch 10/300
Average training loss: 0.0101829907198747
Average test loss: 26.34540851364533
Epoch 11/300
Average

In [ ]:
gauss_10_proj15_model = DAMP(gaussian_10_normalized, 0.01, num_projections, projector, device, residual=False).to(device)
gauss_20_proj15_model = DAMP(gaussian_20_normalized, 0.01, num_projections, projector, device, residual=False).to(device)
gauss_30_proj15_model = DAMP(gaussian_30_normalized, 0.01, num_projections, projector, device, residual=False).to(device)
gauss_40_proj15_model = DAMP(gaussian_40_normalized, 0.01, num_projections, projector, device, residual=False).to(device)

gauss_10_proj15_psnr = average_PSNR(gauss_10_proj15_model, gauss_10_proj15_weights, num_projections, gaussian_10_test, print_psnr=True)
gauss_20_proj15_psnr = average_PSNR(gauss_20_proj15_model, gauss_20_proj15_weights, num_projections, gaussian_20_test, print_psnr=True)
gauss_30_proj15_psnr = average_PSNR(gauss_30_proj15_model, gauss_30_proj15_weights, num_projections, gaussian_30_test, print_psnr=True)
gauss_40_proj15_psnr = average_PSNR(gauss_40_proj15_model, gauss_40_proj15_weights, num_projections, gaussian_40_test, print_psnr=True)

with open(os.path.join(proj15_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_10_proj15_psnr, f)
with open(os.path.join(proj15_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_20_proj15_psnr, f)
with open(os.path.join(proj15_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_30_proj15_psnr, f)
with open(os.path.join(proj15_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_40_proj15_psnr, f)

In [ ]:
# 30 Projections
num_projections = 30

# Folder Path for 30 Projections
proj30_path = 'DAMP_No_Residual-LastLayer/30 Projections'

gauss_10_proj30_weights, gauss_10_proj30_hist = train_main(gaussian_10_normalized, [1] * num_projections, gaussian_10_train, model=projector, device=device, model_type='DAMP', split=0.2, 
                                                     numProjections=num_projections, residual=False, loss_function=loss_function, model_dir='Model_Paths/Gaussian_10_proj30', display=True)

gauss_20_proj30_weights, gauss_20_proj30_hist = train_main(gaussian_20_normalized, [1] * num_projections, gaussian_20_train, model=projector, device=device, model_type='DAMP', split=0.2, 
                                                     numProjections=num_projections, residual=False, loss_function=loss_function, model_dir='Model_Paths/Gaussian_20_proj30', display=True)

gauss_30_proj30_weights, gauss_30_proj30_hist = train_main(gaussian_30_normalized, [1] * num_projections, gaussian_30_train, model=projector, device=device, model_type='DAMP', split=0.2, 
                                                     numProjections=num_projections, residual=False, loss_function=loss_function, model_dir='Model_Paths/Gaussian_30_proj30', display=True)

gauss_40_proj30_weights, gauss_40_proj30_hist = train_main(gaussian_40_normalized, [1] * num_projections, gaussian_40_train, model=projector, device=device, model_type='DAMP', split=0.2, 
                                                     numProjections=num_projections, residual=False, loss_function=loss_function, model_dir='Model_Paths/Gaussian_40_proj30', display=True)

# Store Best Model Weights and History

save_progress(proj30_path, gauss_10_proj30_weights, gauss_10_proj30_hist, '10% Sampling')
save_progress(proj30_path, gauss_20_proj30_weights, gauss_20_proj30_hist, '20% Sampling')
save_progress(proj30_path, gauss_30_proj30_weights, gauss_30_proj30_hist, '30% Sampling')
save_progress(proj30_path, gauss_40_proj30_weights, gauss_40_proj30_hist, '40% Sampling')

In [ ]:
gauss_10_proj30_model = DAMP(gaussian_10_normalized, 0.01, num_projections, projector, device, residual=False).to(device)
gauss_20_proj30_model = DAMP(gaussian_20_normalized, 0.01, num_projections, projector, device, residual=False).to(device)
gauss_30_proj30_model = DAMP(gaussian_30_normalized, 0.01, num_projections, projector, device, residual=False).to(device)
gauss_40_proj30_model = DAMP(gaussian_40_normalized, 0.01, num_projections, projector, device, residual=False).to(device)

gauss_10_proj30_psnr = average_PSNR(gauss_10_proj30_model, gauss_10_proj30_weights, num_projections, gaussian_10_test, print_psnr=True)
gauss_20_proj30_psnr = average_PSNR(gauss_20_proj30_model, gauss_20_proj30_weights, num_projections, gaussian_20_test, print_psnr=True)
gauss_30_proj30_psnr = average_PSNR(gauss_30_proj30_model, gauss_30_proj30_weights, num_projections, gaussian_30_test, print_psnr=True)
gauss_40_proj30_psnr = average_PSNR(gauss_40_proj30_model, gauss_40_proj30_weights, num_projections, gaussian_40_test, print_psnr=True)

with open(os.path.join(proj30_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_10_proj30_psnr, f)
with open(os.path.join(proj30_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_20_proj30_psnr, f)
with open(os.path.join(proj30_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_30_proj30_psnr, f)
with open(os.path.join(proj30_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_40_proj30_psnr, f)